In [1]:
s = "def greeting(name: str) -> str: \
    return 'Hello ' + name "

In [2]:
import astor,ast

In [3]:
a = ast.parse(s)

In [4]:
#!pipenv install typed-ast

In [5]:
from typed_ast import ast3

In [6]:
a = ast3.parse(s)

In [7]:
a

In [8]:
hypothesis_mappings = {"str": {"decorator":"@given(text())","hypothesis_test":"s"}}

def make_test(a):
    for i in a.body:
        name = i.name
        r = i.returns.id
        args = [ x.arg for x in i.args.args]
        args = ", ".join(args)
        hypothesis_test = hypothesis_mappings[r]["hypothesis_test"]
        t = hypothesis_mappings[r]["decorator"]+"\n"
        t += f'def test_{name}({hypothesis_test}):\n'
        t +=f'    assert type({name}({hypothesis_test})) is {r}'
        return t

In [9]:
print(make_test(a))

@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str


In [10]:
#!pipenv install hypothesis

In [11]:
from hypothesis import given
from hypothesis.strategies import text

def greeting(name: str) -> str: 
    return 'Hello ' + name 
@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str

In [12]:
test_greeting()

In [13]:
data={"datasets": [{"label": "data1", "data": [{"x": 1, "y": 1}, {"x": 2, "y": 2}]}]}

In [14]:
s=open("t_t.py").read()
a = ast3.parse(s)

In [15]:
a.body

In [16]:
import json
from collections import namedtuple

In [44]:
from typing import List
class DataPoint:
    def __init__(self,x:int,y:int):
        self.x = x
        self.y =y

class DataSet:
    def __init__(self,label:str,data:List[DataPoint],backgroundColor:str):
        self.label=label
        self.data=data
        self.backgroundColor = backgroundColor

class Data:
    def __init__(self,datasets:List[DataSet]=None):
        self.datasets=datasets

In [126]:
data= {"datasets": [{"label": "data1", "data": [{"x": 1.0, "y": 1.0}, {"x": 2.0, "y": 2.0}]}]}
data1 = Data([DataSet(label="label1",data=[DataPoint(1,2),DataPoint(3,4)],backgroundColor="00b8ff")])

In [127]:
from crawto.Charts.charts import tsne_plot

In [128]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
tsne_plot(data1)